## Unsupervised Learning

Unsupervised learning consists of the set of statistical tools that are used in the setting in which we have only a set of $p$ features measured on $n$ observations, but we do not have an associated response variable $Y$ - that is, there are no training labels. The goal is to discover interesting things about the measurements on $X_1, X_2,\dots, X_p$. The two main types of unsupervised are clustering and dimensionality reduction. We will focus on two types of clustering, k-means and hierarchical clustering, as well as the dimensionality tool principal components analysis, which is widely used for preprocessing and data visualization. 

### The Challenge of Unsupervised Learning

Unsupervised learning is often more challenging than supervised learning. It is a more subjective type of analysis, as there is no simple goal, as is often the case in supervised learning. Unsupervised learning is often used in exploratory data analysis. 

The main difference is this: in supervised learning, we have universally accepted mechanisms for checking our work, cross validation or validating results on an independent data set. We can see how our predictive model works on observations not used in fitting the model. In unsupervised learning, there is no way to check our work because there is no true answer - the problem is unsupervised and open ended. 

## Clustering

Clustering is an unsupervised method that tries to find structure in a set of objects. Specifically, we want to find clusters, groups within our object set where objects in a cluster are more similar to each other than they are to objects outside of the cluster. Note that this definition is quite vague - there are many different ways to conceptualize how clusters may occur in data, and today we will look at K-Means and hierarchical clustering in particular.

### K-Means Clustering

Given a set of objects, we specify that we want $k$ clusters. Each cluster has a mean representing it, and we assign each point to the clusters based on which cluster mean its closest to. For each point, the reconstruction error is defined to be its distance to its cluster mean. This gives us the total reconstruction error, the sum of all the individual reconstruction errors, as an error value that we want to minimize. 

Formally, we have a set of object vectors $\{x_n\}_{n = 1}^N$ and a set of $K$ cluster means $\{\mu_k\}_{k = 1}^K$, where $x_n, \mu_k \in \mathbb{R}^D$. We represent each object's cluster assignment with the $K$ dimensional vector $r_n$, where $r_{nk} = 1$ if $x_n$ is in cluster $k$, and 0 otherwise. This gives us the individual reconstruction error

$$J_n(r_n, \{\mu_k\}) = \sum_{k = 1}^K r_{nk} \cdot |x_n - \mu_k|^2$$

and the total reconstruction error 

$$J(\{r_n\}, \{\mu_k\}) = \sum_{n = 1}^N \sum_{k = 1}^K r_{nk} \cdot |x_n - \mu_k|^2$$

As you can see, the reconstruction error on a set of objects is a function of assignments and means. How would you go about choosing the assignments $\{r_n\}$ and means $\{\mu_k\}$ that minimize the reconstruction error? Lloyd's Algorithm proposes a two step error minimization.

**Step 1**: minimize $J$ by updating the $r_n$, assigning each $x_n$ to its closest cluster mean.

**Step 2**: minimize $J$ by recalculting the $\mu_k$ to be the average over all vectors assigned to cluster $k$.

Repeating this process until the assignments do not change, the algorithm will converge upon a local minima. The algorithm can be optimized by starting with reasonable distributions for cluster centers rather than choosing randomly (K-Means ++) or adding the condition that the cluster mean must be a point (K-Medoids). 

### Hierarchical Clustering

K-Means is one of the most common and simple clustering methods, but it has a couple of key limitations. First off, it is nondeterministic as it depends on the initial choice of cluster means, and Lloyd's Algorithm only arrives upon local minima rather than the global minimum. Furthermore, the algorithm requires you to decide what $k$ is, as we saw earlier. Finally, K-Means can be inflexible, as the only way cluster centers are derived is through the mean distance from all of the points assigned to the cluster. Because of this construction, K-Means doesn't perform well on clusters that are connected but not compact.

Hierarchical clustering solves many of these issues. The motivation between hierarchical clustering is building up clusters as a hierarchy. All objects start out in their individual groups, and at each level, the groups that are a certain distance apart are joined to form a larger group. A variety of different distance metrics can be used in building up these groups to result in different types of hierarchical clusters. 

![Dendrogram Example](http://i.stack.imgur.com/x5UKu.png)

A dendrogram is a way of visualizing the groups as they are aggregated together in the hierarchy. As you can see, hierarchical clustering not only resolves some of the problems explained concerning K-Means - it also prevents a very nice way of representing the structure of the clusters, identifying subclusters within the clusters.

## PCA

### Dimensionality Reduction?

Sometimes when we're analyzing data, we might find that the data is very highly dimensional, and we don't really know how to work with it. What techniques do we have for understanding very highly dimensional data? 

We could:
1. Use a random forest, which is designed for very highly dimensional data.
2. Try to fit a linear model to this.
3. Use an SVM, which also handles high dimensional data well.

So we actually have a lot of tools that are pretty robust on highly dimensional data! Unfortunately, our visualization skills are not necessarily up to par - even if we do get a great output from our models, we want to be able to interpret all of our data. Also, we might have hundreds or even thousands of useless, noisy features that we want to delete from our model because they either don't vary much, or are explained by other features.

So, we can use **Principle Components Analysis (PCA)** as a *dimensionality-reduction* technique. We can use it in the early stages of our pipeline to reduce the dimensionality of our data to make it easier to process, visualize, and think about. Let's see how it works.

### Formalizing The Problem

Basically, the idea behind PCA is that our data points $x_1, x_2, x_3, \ldots, x_N$ are tuples of highly dimensional data (so they're vectors in ${\mathbb R}^m$, where $m$ is a large number. We want to *summarize* this data with vectors $\phi_1, \phi_2, \ldots \phi_N \in {\mathbb R}^n$, where $n$ is much smaller than $m$. What does good summarization mean?

Let's say that the map from ${\mathbb R}^m \to {\mathbb R}^n$ is given by a matrix $M$ (we're looking for linear maps - notably, PCA is a *linear* dimensionality reduction technique. There are many nonlinear techniques, but we won't discuss them here). So, we know that $M$ is an $n \times m$ matrix, where we take a regular point $x_i \to Mx_i = \phi_i$. Then the columns of $M$ are the basis vectors of the transformation, but what are the rows? We're looking to find the ability of the map $M$ to summarize the data points $x_i$. 


### PCA in Principle (optional)

One way to do this is to look at the row vectors of $M$, call them $\alpha_i$, and think of them as a partial basis of the space of all vectors. If we have $n = m$, then we can have a full basis, and the regular basis $e_1 = (1, 0, 0, \ldots), \ldots$ is perfectly fine, but since we're trying to do dimensionality reduction, we want to think about how much data we *can't* explain, or is contained in the complement subspace to the one we are looking at. Suppose our row vectors are $\alpha_i$, and suppose furthermore that they're orthonormal (orthogonality is crucial because we want as efficient a subspace as possible, and normality doesn't change our ability to explain the results). Then, the amount of *stuff in $x_i$* that can't be explained by the subspace is:
$$E_{\text{PCA}} = \sum_{i = 1}^N ||x_i - \sum_{j}(\alpha_j \cdot x_i)x_i||^2$$

where we're taking the dot product of $x_i$ with the basis, adding that together to form the *reconstructed* vector ($\sum_{j}(\alpha_j \cdot x_i)x_i$), and then finding the distance between that and the original, and summing the squares of those distances over all the vectors. We want to minimize this quantity.

It turns out that minimizing this quantity is equivalent to maximizing the covariance of the subspace, and we can find that by finding the top $n$ eigenvectors from the $m \times m$ covariance matrix of the data points:
$$C = \frac{1}{N}\sum_{i = 1}^N x_ix_i^T$$

So, once we extract the eigenvectors from this matrix, we can go ahead and build the matrix $M$, and then just map the points over as $\phi_i = Mx_i$, and we have our new 

How do we calibrate the value of $n$? In essence, how many dimensions can we reasonably drop from our data? While the ultimate answer will depend on your goals, one way to figure out what's going on is to look at *eigenspectrum*. This the list of the eigenvalues of $C$ in order, from largest to smallest. If 90% of the total sum of the eigenvalues is contributed by a few variables, those variables might be a good idea to keep, while dropping the rest. 